# STAT 542 / CS 598: Homework 4

Fall 2019, by Edgar Pino

Due: Monday, Oct 14 by 11:59 PM Pacific Time

In [120]:
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

# Question 1 [70 Points] Tuning Random Forests in Virtual Twins

In [3]:
health_df = pd.read_csv('./data/sepsis.csv')

In [4]:
health_df = health_df.drop(columns=['Unnamed: 0'])

In [5]:
health_df.sample(10)

,Health,THERAPY,PRAPACHE,AGE,BLGCS,ORGANNUM,BLIL6,BLLPLAT,BLLBILI,BLLCREAT,TIMFIRST,BLADL,blSOFA,BEST
170,-0.251315,1,19,49.823,15,2,723.000000,45.0,3.4,1.0,10.00,0.0,10.0,1
149,0.193811,1,28,53.656,14,1,1232.000000,650.0,0.4,1.0,10.00,8.0,3.0,0
407,-4.317473,1,19,56.098,12,2,406.600000,244.0,0.9,1.0,3775.90,0.0,6.0,0
119,6.779147,1,25,68.818,13,1,296550.000000,137.0,0.4,1.4,13.78,9.0,7.0,0
59,-1.664960,0,19,53.656,11,2,15844.532433,172.0,1.0,1.0,50.67,0.0,7.0,0
332,-1.399295,0,19,73.138,15,1,118.900000,650.0,0.8,1.1,39.83,0.0,7.0,0
244,0.448185,0,29,49.823,10,3,406.600000,153.0,2.5,1.8,21.73,0.0,12.0,1
103,-0.377317,1,27,63.471,12,1,519.400000,45.0,3.4,2.2,50.67,0.0,13.0,0
285,2.504733,0,22,56.098,15,4,92.800000,78.0,3.4,3.8,30.67,0.0,17.0,0
305,-1.678036,1,21,68.818,15,2,4952.000000,137.0,0.4,1.0,34.83,0.0,8.0,0


In [6]:
health_active = health_df.loc[health_df['THERAPY'] == 1]

In [7]:
health_control = health_df.loc[health_df['THERAPY'] == 0]

## Randomly split the data into 75% for training and 25% for testing.

In [8]:
y_active = health_active.Health

In [9]:
x_active = health_active.drop(columns=['Health'])

In [10]:
y_control = health_control.Health

In [11]:
x_control = health_control.drop(columns=['Health'])

In [105]:
X_active_train, X_active_test, y_active_train, y_active_test = train_test_split(x_active, y_active, test_size=0.25)

In [106]:
X_control_train, X_control_test, y_control_train, y_control_test = train_test_split(x_control, y_control, test_size=0.25, )

## For the training data, fit the virtual twins model and then use the testing data to suggest the best treatment.
- You should not use the variable BEST when fitting the models
- Pick three different mtry values and three different nodesize, leave all other tuning parameters as default
- After predicting the best treatment in the testing data, compare it to the truth BEST


In [117]:
max_features_vals = [5, 10, 15]

In [118]:
min_samples_leaf = [1,3,5]

In [126]:
params = []

In [127]:
for r in itertools.product(max_features_vals, min_samples_leaf): 
    params.append([r[0],r[1]])

In [107]:
active_regresssor = RandomForestRegressor(max_features=5, min_samples_leaf=1).fit(X_active_train.drop(columns=['BEST', 'THERAPY']), y_active_train)

In [108]:
control_regresssor = RandomForestRegressor(max_features=5, min_samples_leaf=1).fit(X_control_train.drop(columns=['BEST', 'THERAPY']), y_control_train)

In [109]:
x_test = pd.concat([X_active_test, X_control_test])

In [110]:
y_active_pred = active_regresssor.predict(x_test.drop(columns=['BEST', 'THERAPY']))

In [111]:
y_control_pred = control_regresssor.predict(x_test.drop(columns=['BEST', 'THERAPY']))

In [112]:
y_pred = []

In [113]:
for i in range(len(y_test)):
    if y_active_pred[i] > y_control_pred[i]:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [114]:
accuracy = accuracy_score(x_test.BEST, y_pred)

In [116]:
accuracy

0.7394957983193278